## Pipeline qiime2 BLAST des OTUs sur amplicons Illumina

* importation des données DADA2
Séquences illumina avec plusieurs marqueurs par index, ici 16SV4 et ITS2.
On débruite et déréplique tout en bloc avec DADA2 sous R car c'est beaucoup plus rapide que DADA2 sous environnement qiime2. Puis on importe la table des séquences uniques de DADA2 en artefacts qiime2.

**sous R (DADA2):**
write.table(t(seqtab.nochim), "~/sync/mangroves/data_sequencages/guyane/dada2-analysis/seqtab-nochim.txt", sep="\t", row.names=TRUE, col.names=NA, quote=FALSE)

uniquesToFasta(seqtab.nochim, "~/sync/mangroves/data_sequencages/guyane/dada2-analysis/rep-seqs.fna", ids=colnames(seqtab.nochim))

* sous bash, activation environnement conda qiime2

**Le trimming des amorces (15pb à gauche, 15pb à droite) permet d'accroitre d'environ 30% le nombre de reads filtrés, en diminuant le nombre de chimères**

--> sans trimmer les amorces
"input" "filtered" "denoisedF" "denoisedR" "merged" "nonchim"
"Blancs" 24460 6389 5785 5461 3793 2234
"S1D101" 85622 58201 44011 42027 9794 8924
"S1D102" 99085 65076 52672 50128 11480 10160
"S1D203" 64106 46813 37450 36485 10696 9727
"S1D204" 66110 46613 37775 36016 11683 10316
"S1D205" 91809 63124 50219 49882 14095 12630


--> en trimmant les amorces
"input" "filtered" "denoisedF" "denoisedR" "merged" "nonchim"
"Blancs" 24460 6936 6377 6136 4707 4110
"S1D101" 85622 61596 52863 54327 14146 13034
"S1D102" 99085 68605 61354 61952 18091 16729
"S1D203" 64106 49032 43066 43950 14611 13768
"S1D204" 66110 48630 42887 43691 15766 14586
"S1D205" 91809 65849 57067 59604 18494 17002

* il faut au préalable créer les dossiers /qiime2, /OTU-ITS, /OTU-16S et OTU-16S-taxo
* création de l'artefact rep-seqs.qza issu de seqtab.nochim (renommé en rep-seqs.fna sous DADA2/R)

In [ ]:
source activate qiime2-amplicon-2023.9
##conda info --envs

In [ ]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024

qiime tools import \
--input-path ./dada2/seqtabnochim_guadeloupe_LB_final_mai24_notrim.fna \
--type 'FeatureData[Sequence]' \
--output-path ./qiime2/rep-seqs_guadeloupe_LB_16S_18S_ITS.qza

* création de l'artefact table.qza par biom

In [ ]:
# echo -n "#OTU Table" | cat -  > ./biom-table_guyane_all.txt
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024

biom convert -i ./dada2/seqtabnochim_guadeloupe_LB_final_mai24_notrim.txt -o ./qiime2/table_guadeloupe_LB_16S_18S_ITS.biom --table-type="OTU table" --to-hdf5

qiime tools import \
--input-path ./qiime2/table_guadeloupe_LB_16S_18S_ITS.biom \
--type 'FeatureTable[Frequency]' \
--output-path ./qiime2/table_guadeloupe_LB_16S_18S_ITS.qza

In [ ]:
### dans DADA2 on a trimmé 15bp des deux côtés des reads pour diminuer le taux de chimères
### du coup on effectue Vsearch et on assigne directement

### ou alors on ne trimme rien et on trie par marqueur, avant Vsearch et assignation (mais on perd plus de la moitié des reads)

In [ ]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime feature-classifier extract-reads \
  --i-sequences rep-seqs_guadeloupe_LB_16S_18S_ITS.qza \
  --p-f-primer GTGYCAGCMGCCGCGGTAA \
  --p-r-primer CCGYCAATTYMTTTRAGTTT \
  --p-identity 0.7 \
  --p-n-jobs 6 \
  --o-reads ./OTU-16S/rep-seqs_guadeloupe_LB_16S.qza
  
qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_16S_18S_ITS.qza \
  --m-metadata-file ./OTU-16S/rep-seqs_guadeloupe_LB_16S.qza \
  --o-filtered-table ./OTU-16S/table_guadeloupe_LB_16S.qza

In [ ]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime feature-classifier extract-reads \
  --i-sequences rep-seqs_guadeloupe_LB_16S_18S_ITS.qza \
  --p-f-primer CCCTGCCHTTTGTACACAC \
  --p-r-primer CCTTCYGCAGGTTCACCTAC \
  --p-identity 0.7 \
  --p-n-jobs 6 \
  --o-reads ./OTU-18S/rep-seqs_guadeloupe_LB_18S.qza
  
qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_16S_18S_ITS.qza \
  --m-metadata-file ./OTU-18S/rep-seqs_guadeloupe_LB_18S.qza \
  --o-filtered-table ./OTU-18S/table_guadeloupe_LB_18S.qza

In [ ]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime feature-classifier extract-reads \
  --i-sequences rep-seqs_guadeloupe_LB_16S_18S_ITS.qza \
  --p-f-primer GTGAATCATCGAATCTTTGAA \
  --p-r-primer TCCTCCGCTTATTGATATGC \
  --p-identity 0.9 \
  --p-n-jobs 6 \
  --o-reads ./OTU-ITS/rep-seqs_guadeloupe_LB_ITS.qza
  
qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_16S_18S_ITS.qza \
  --m-metadata-file ./OTU-ITS/rep-seqs_guadeloupe_LB_ITS.qza \
  --o-filtered-table ./OTU-ITS/table_guadeloupe_LB_ITS.qza

* VSEARCH produit la table des OTUs (<3%), pour chaque marqueur

In [ ]:
## pour obtenir le fasta il faut ouvrir le rep-seqs_XXX.qza avec un utilitaire d'archives
## extraire le fasta du dossier data
## et le renommer en rep-seqs_XXX.fasta

cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-16S/

vsearch --cluster_size rep-seqs_guadeloupe_LB_16S.fasta \
    --threads 8 \
    --id 0.97 \
    --sizeout \
    --fasta_width 0 \
    --qmask soft \
    --centroids rep-seqs-97_guadeloupe_LB_16S.fasta \
    --otutabout rep-seqs-97_guadeloupe_LB_16S.txt

qiime tools import \
--input-path rep-seqs-97_guadeloupe_LB_16S.fasta \
--type 'FeatureData[Sequence]' \
--output-path rep-seqs-97_guadeloupe_LB_16S.qza

qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_16S.qza \
  --m-metadata-file rep-seqs-97_guadeloupe_LB_16S.qza \
  --o-filtered-table table-97_guadeloupe_LB_16S.qza

In [ ]:
## d'abord extraire le fasta comme indiqué cellulle plus haut
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-18S/

vsearch --cluster_size rep-seqs_guadeloupe_LB_18S.fasta \
    --threads 8 \
    --id 0.97 \
    --sizeout \
    --fasta_width 0 \
    --qmask soft \
    --centroids rep-seqs-97_guadeloupe_LB_18S.fasta \
    --otutabout rep-seqs-97_guadeloupe_LB_18S.txt

qiime tools import \
--input-path rep-seqs-97_guadeloupe_LB_18S.fasta \
--type 'FeatureData[Sequence]' \
--output-path rep-seqs-97_guadeloupe_LB_18S.qza

qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_18S.qza \
  --m-metadata-file rep-seqs-97_guadeloupe_LB_18S.qza \
  --o-filtered-table table-97_guadeloupe_LB_18S.qza

In [ ]:
## d'abord extraire le fasta comme indiqué cellulle plus haut
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-ITS/

vsearch --cluster_size rep-seqs_guadeloupe_LB_ITS.fasta \
    --threads 8 \
    --id 0.97 \
    --sizeout \
    --fasta_width 0 \
    --qmask soft \
    --centroids rep-seqs-97_guadeloupe_LB_ITS.fasta \
    --otutabout rep-seqs-97_guadeloupe_LB_ITS.txt

qiime tools import \
--input-path rep-seqs-97_guadeloupe_LB_ITS.fasta \
--type 'FeatureData[Sequence]' \
--output-path rep-seqs-97_guadeloupe_LB_ITS.qza

qiime feature-table filter-features \
  --i-table table_guadeloupe_LB_ITS.qza \
  --m-metadata-file rep-seqs-97_guadeloupe_LB_ITS.qza \
  --o-filtered-table table-97_guadeloupe_LB_ITS.qza

* exporter les tables d'OTUs

In [ ]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime tools export \
    --input-path ./OTU-16S/table-97_guadeloupe_LB_16S.qza \
    --output-path ./OTU-16S
    
biom convert \
    -i ./OTU-16S/feature-table.biom \
    -o ./OTU-16S/feature-table-97_guadeloupe_LB_16S.tsv --to-tsv


qiime tools export \
    --input-path ./OTU-18S/table-97_guadeloupe_LB_18S.qza \
    --output-path ./OTU-18S
    
biom convert \
    -i ./OTU-18S/feature-table.biom \
    -o ./OTU-18S/feature-table-97_guadeloupe_LB_18S.tsv --to-tsv

qiime tools export \
    --input-path ./OTU-ITS/table-97_guadeloupe_LB_ITS.qza \
    --output-path ./OTU-ITS
    
biom convert \
    -i ./OTU-ITS/feature-table.biom \
    -o ./OTU-ITS/feature-table-97_guadeloupe_LB_ITS.tsv --to-tsv

# ouvrir le .tsv avec LibreOffice Calc, sélectionner les abondances, Edition>Find and Replace,
# chercher ^[0-9] et remplacer par & (cocher "regular expressions")

In [ ]:
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2

qiime tools export \
    --input-path ./OTU-ITS/table-97_guadeloupe_LB_ITS.qza \
    --output-path ./OTU-ITS
    
biom convert \
    -i ./OTU-ITS/feature-table.biom \
    -o ./OTU-ITS/feature-table-97_guadeloupe_LB_ITS.tsv --to-tsv

* feature-classifier BLAST
* de préférence sur un cluster / un calculateur

In [ ]:
#########################
## UNITE
## import de la base en artefact .qza
## source activate qiime2-amplicon-2023.9
cd /Volumes/BOUGREDANE/references_metabarcoding/ITS2/sh_qiime_release_s_all_04.04.2024
#mkdir ./qiime

qiime tools import \
  --type FeatureData[Sequence] \
  --input-path ./sh_refs_qiime_ver10_97_s_all_04.04.2024.fasta \
  --output-path ./sh_refs_qiime_ver10_97_s_all_04.04.2024.qza
  
qiime tools import \
  --type FeatureData[Taxonomy] \
  --input-path ./sh_taxonomy_qiime_ver10_97_s_all_04.04.2024.txt \
  --output-path ./sh_taxonomy_qiime_ver10_97_s_all_04.04.2024.qza
  
  #  --input-format HeaderlessTSVTaxonomyFormat \

In [ ]:
####################################
### ITS2 : Unite
## source activate qiime2-amplicon-2023.9
## mkdir ./Unite10

cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-ITS

qiime feature-classifier classify-consensus-blast \
    --i-query ./rep-seqs-97_guadeloupe_LB_ITS.qza \
    --i-reference-reads /Users/tonyrobinet/sync/references_metabarcoding/ITS2/sh_refs_qiime_ver10_97_s_all_04.04.2024.qza \
    --i-reference-taxonomy /Users/tonyrobinet/sync/references_metabarcoding/ITS2/sh_taxonomy_qiime_ver10_97_s_all_04.04.2024.qza \
    --o-classification ./Unite10/guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza \
    --o-search-results ./Unite10/top_hits_guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza

# exporter en .biom
qiime tools export \
    --input-path ./Unite10/guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza \
    --output-path ./Unite10


In [ ]:
conda deactivate
source activate qiime2-shotgun-2023.9
cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-ITS

qiime metadata tabulate \
--m-input-file rep-seqs-97_guadeloupe_LB_ITS.qza \
--m-input-file table_97_guadeloupe_LB_ITS.qza \
--m-input-file Unite10/guadeloupe_LB_ITS_Unite_ver10_97_s_all_04.04.2024.qza \
--o-visualization merged.qzv

qiime tools export \
  --input-path merged.qzv \
  --output-path merged-data

In [ ]:
# BLAST+ 18S
# on aura préalablement entraîné le classifieur sur les amorces 18SV9 à partir de la base Silva 138.1
# voir le script classifier_rescript_18SV9.ipynb

cd ~/sync/mangroves/data_sequencages/guadeloupe/2024/qiime2/OTU-18S

qiime feature-classifier classify-sklearn \
   --i-reads ./rep-seqs-97_guadeloupe_LB_18S.qza \
   --i-classifier /Users/tonyrobinet/sync/references_metabarcoding/Silva138.1/silva-138.1-ssu-nr99-18SV9-classifier.qza \
   --p-n-jobs -1 \
   --output-dir ./OTU-18S-taxo

# exporter en .biom
qiime tools export \
    --input-path ./OTU-18S-taxo/classification.qza \
    --output-path ./OTU-18S-taxo

** On a ainsi créé, par marqueur :
- un tableau des OTUs à 3% (séquences et abondances des reads après filtres qualités) [feature-table_guyane_ITS2.tsv]
- un tableau des assignations taxonomiques des OTUs [taxonomy.tsv]